In [1]:
import pandas as pd
from sklearn import neighbors
import matplotlib.pyplot as plt
import numpy as np

import dataPrepare as dp
from FeatureEngineeringManualRun import getFeatures

from sklearn.ensemble import GradientBoostingRegressor

c:\users\sebastian\appdata\local\programs\python\python36\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from pprint import pprint
from time import time
from sklearn.metrics import classification_report
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
import logging


In [3]:
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import math

In [4]:
dfx = dp.getData(200000)
target = dp.target


dfx = getFeatures(dfx)

testDf = dp.getValidData(30000)
testDf = getFeatures(testDf)
df = dfx.drop(columns=dp.categorical_cols)
features = list(df.columns)
features.remove(dp.target)
x_train = df[features]
y_train = df[target]
x_test = testDf[features]
y_test = testDf[target]

In [5]:
clf = GradientBoostingRegressor(n_estimators=200,min_samples_leaf=2,max_depth=3,verbose=1)
clf.fit(x_train, y_train)

      Iter       Train Loss   Remaining Time 
         1           0.0799            2.94m
         2           0.0682            3.14m
         3           0.0586            3.23m
         4           0.0509            3.24m
         5           0.0445            3.22m
         6           0.0393            3.16m
         7           0.0349            3.05m
         8           0.0313            2.95m
         9           0.0284            2.91m
        10           0.0259            2.89m
        20           0.0154            2.76m
        30           0.0125            2.59m
        40           0.0113            2.39m
        50           0.0107            2.18m
        60           0.0102            2.06m
        70           0.0100            1.86m
        80           0.0096            1.71m
        90           0.0094            1.56m
       100           0.0091            1.40m
       200           0.0080            0.00s


GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=2, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=200,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=1, warm_start=False)

In [6]:
y_pred = clf.predict(x_test)

In [7]:
error_MAE = mean_absolute_error(y_test,y_pred) #calculate err
print('MAE value  is:', error_MAE)
error_RMSE = math.sqrt(mean_squared_error(y_test,y_pred)) #calculate err
print('RMSE value  is:', error_RMSE)
r2 = r2_score(y_test, y_pred)
print('R2: ', r2)

MAE value  is: 0.06458851237584845
RMSE value  is: 0.09146371292670824
R2:  0.9118321874966576


In [ ]:
clf_2 = GradientBoostingRegressor(n_estimators=250,min_samples_leaf=100,max_depth=7,verbose=1)
clf_2.fit(x_train, y_train)

      Iter       Train Loss   Remaining Time 
         1           0.0787           18.82m
         2           0.0660           18.99m
         3           0.0557           18.80m
         4           0.0473           18.66m
         5           0.0405           18.47m
         6           0.0348           18.35m
         7           0.0303           18.26m
         8           0.0265           17.97m
         9           0.0234           17.94m
        10           0.0208           17.83m
        20           0.0102           17.23m
        30           0.0082           16.03m
        40           0.0075           14.91m
        50           0.0071           13.59m
        60           0.0069           12.30m
        70           0.0068           11.20m
        80           0.0067           10.15m
        90           0.0066            9.42m
       100           0.0066            8.56m


In [ ]:
y_pred_2 = clf_2.predict(x_test)

In [ ]:
error_MAE_2 = mean_absolute_error(y_test,y_pred_2) #calculate err
print('MAE value  is:', error_MAE_2)
error_RMSE_2 = math.sqrt(mean_squared_error(y_test,y_pred_2)) #calculate err
print('RMSE value  is:', error_RMSE_2)
r2_2 = r2_score(y_test, y_pred_2)
print('R2: ', r2_2)

In [11]:
tunning_x_train = x_train.head(20000)
tunning_y_train = y_train.head(20000)

In [14]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [16]:
# specify pipeline
pipeline = Pipeline([
    ('clf', GradientBoostingRegressor()),
])

# add more parameters will give better exploring power but will
# increase processing time in a combinatorial way
parameters = {
    #TUTAJ TWORZYMY LISTY PARAMETRÓW WEDŁUG WZORCA
     #<nazwa zmiennej z pipeline>__<nazwa parametru>
#     'clf__n_estimators': (100,200,250,300,400,500,600,700,800),
    'clf__n_estimators': (100,200,250,300,400),
    'clf__max_depth' :(1,2,3,5,7),
    'clf__min_samples_leaf':(1,10,100,0.001,0.0001),
#     'clf__max_features':('auto','sqrt','log2',None),

    #Choosing max_features < n_features leads to a reduction of variance and an increase in bias.
}

# specify metrics
scores = {
#     'max_error': make_scorer(mean_absolute_error),
#     'neg_mean_absolute_error': make_scorer(mean_absolute_error),
    'neg_mean_squared_error': make_scorer(mean_squared_error),
#     'neg_mean_squared_log_error': make_scorer(mean_squared_log_error),
#     'neg_median_absolute_error': make_scorer(median_absolute_error),
#     'r2': make_scorer(r2_score),
}


In [17]:
#Poszukiwanie paramterów
for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    grid_search = GridSearchCV(pipeline, parameters,
                       scoring=score,n_jobs=-1, cv=5, verbose=1)
    grid_search.fit(tunning_x_train, tunning_y_train)
    report(grid_search.cv_results_, n_top=5)

# Tuning hyper-parameters for neg_mean_squared_error

Fitting 5 folds for each of 125 candidates, totalling 625 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 625 out of 625 | elapsed: 16.7min finished


Model with rank: 1
Mean validation score: -0.008 (std: 0.000)
Parameters: {'clf__max_depth': 7, 'clf__min_samples_leaf': 100, 'clf__n_estimators': 250}

Model with rank: 2
Mean validation score: -0.008 (std: 0.000)
Parameters: {'clf__max_depth': 7, 'clf__min_samples_leaf': 100, 'clf__n_estimators': 200}

Model with rank: 3
Mean validation score: -0.008 (std: 0.000)
Parameters: {'clf__max_depth': 5, 'clf__min_samples_leaf': 100, 'clf__n_estimators': 400}

Model with rank: 4
Mean validation score: -0.008 (std: 0.000)
Parameters: {'clf__max_depth': 7, 'clf__min_samples_leaf': 0.001, 'clf__n_estimators': 200}

Model with rank: 5
Mean validation score: -0.008 (std: 0.000)
Parameters: {'clf__max_depth': 7, 'clf__min_samples_leaf': 100, 'clf__n_estimators': 300}



In [15]:
report(grid_search.cv_results_, n_top=5)

Model with rank: 1
Mean validation score: -0.063 (std: 0.001)
Parameters: {'clf__max_depth': 7, 'clf__min_samples_leaf': 100, 'clf__n_estimators': 250}

Model with rank: 2
Mean validation score: -0.063 (std: 0.001)
Parameters: {'clf__max_depth': 7, 'clf__min_samples_leaf': 100, 'clf__n_estimators': 200}

Model with rank: 3
Mean validation score: -0.063 (std: 0.001)
Parameters: {'clf__max_depth': 7, 'clf__min_samples_leaf': 100, 'clf__n_estimators': 300}

Model with rank: 4
Mean validation score: -0.063 (std: 0.001)
Parameters: {'clf__max_depth': 7, 'clf__min_samples_leaf': 0.001, 'clf__n_estimators': 200}

Model with rank: 5
Mean validation score: -0.063 (std: 0.001)
Parameters: {'clf__max_depth': 5, 'clf__min_samples_leaf': 1, 'clf__n_estimators': 400}



In [ ]:
# specify parameters and distributions to sample from
# add more parameters will give better exploring power but will
# dicrease chance to find optimal combinadtion in constatnt number of itters
# increase number of itets increase processing time in linear way
param_dist = {
                "clf__n_estimators": sp_randint(1, 2000),
                'clf__max_depth' : sp_randint(1, 20),
                "clf__max_features": ['auto','sqrt','log2',None],
                "clf__min_samples_leaf": sp_randint(1, 1000),
             }
# random search iters number
n_iter_search = 1000

In [ ]:
# for score in scores:
#     print("# Tuning hyper-parameters for %s" % score)
#     random_search = RandomizedSearchCV(pipeline, param_distributions=param_dist,scoring=score,
#                                        n_iter=n_iter_search, cv=5, n_jobs=8, verbose=1,iid=False)

#     random_search.fit(tunning_x_train, tunning_y_train)
#     report(random_search.cv_results_)